In [ ]:
import os
import pandas as pd
import re

# 1. 경로 설정
input_folder = '../2.혼잡도/4_same_weekend_data'
output_folder = '../2.혼잡도/5_same_hour_data'
os.makedirs(output_folder, exist_ok=True)

# 2. 파일 반복 처리
for filename in os.listdir(input_folder):
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, filename)
    print(f"📂 처리 중: {filename}")

    try:
        df = pd.read_csv(file_path, encoding="euc-kr")
    except Exception as e:
        print(f"❌ 파일 읽기 실패: {e}")
        continue

    # 3. 시간 컬럼 추출
    time_cols = [col for col in df.columns if re.match(r"\d{2}시\d{2}분", col)]

    # 4. 시 단위로 그룹화
    hour_groups = {}
    for col in time_cols:
        hour = col[:2] + "시"  # 예: "05시30분" -> "05시"
        hour_groups.setdefault(hour, []).append(col)

    # 5. 평균 계산
    for hour, cols in hour_groups.items():
        df[hour] = df[cols].mean(axis=1)

    # 6. 기존 시간 컬럼 제거
    df.drop(columns=time_cols, inplace=True)

    # 7. 열 순서 정리 (기본 정보 + 시 컬럼)
    base_cols = ["평일주말", "호선", "역번호", "역명", "구분"]
    hour_cols = sorted([col for col in df.columns if col not in base_cols])
    df = df[base_cols + hour_cols]

    # 8. 저장
    output_path = os.path.join(output_folder, f"hourlyavg_{filename}")
    try:
        df.to_csv(output_path, index=False, encoding="euc-kr")
        print(f"✅ 저장 완료: {output_path}\n")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
